# EMG analysis

<!--# Plotting data
To plot data, the key of the data dictionary to be plotted must be used. The key actually is a file name for each set of data. There are 4 types of data for now which are **RAW, RECTIFY, MOVING AVERAGE** and **ROOT MEAN SQUARE**. Their names are listed as following:

| For RAW | For RECTIFY | For MOVING AVERAGE | For ROOT MEAN SQUARE |
|:--------------------------:|:------------------------------:|:------------------------------:|:-----------------------------:|
| both_*[subjectname]*_raw.p | both_*[subjectname]*_rectify.p | both_*[subjectname]*_64_mAvg.p | both_*[subjectname]*_64_rms.p |
| roll_*[subjectname]*_raw.p | roll_*[subjectname]*_rectify.p | roll_*[subjectname]*_64_mAvg.p | roll_*[subjectname]*_64_rms.p |
| yaw_*[subjectname]*_raw.p | yaw_*[subjectname]*_rectify.p | yaw_*[subjectname]*_64_mAvg.p | yaw_*[subjectname]*_64_rms.p |

### For RAW
- both_*[subjectname]*_raw.p
- roll_*[subjectname]*_raw.p
- yaw_*[subjectname]*_raw.p

### For RECTIFY
- both_*[subjectname]*_rectify.p
- roll_*[subjectname]*_rectify.p
- yaw_*[subjectname]*_rectify.p

### For MOVING AVERAGE
- both_*[subjectname]*_64_mAvg.p
- roll_*[subjectname]*_64_mAvg.p
- yaw_*[subjectname]*_64_mAvg.p

### For ROOT MEAN SQUARE
- both_*[subjectname]*_64_rms.p
- roll_*[subjectname]*_64_rms.p
- yaw_*[subjectname]*_64_rms.p

*__Note__ this is supposed to help in copy n paste* -->

In [ ]:
# PLOTTING
subjectToPlot = "PA"
axes = "roll"
# tmp = ("roll", "yaw", "both")
# axesn = tmp.index(axes)
# whatToPlot = slow_with_motor[subjectToPlot][axesn] # specific event
whatToPlot = slice(None) # all

# for i in range(10): # 10 channels of data
#     plt.figure(figsize=(40,10)) # (75,15) works fine too
#     plt.title(header[i])
# #     plt.plot(raw_data[axes+'_'+str.lower(subjectName)+'_raw.p'][:,i], label="raw")
# #     plt.plot(rect_data[axes+'_'+str.lower(subjectName)+'_rectify.p'][:,i], label="rect")
#     plt.plot(mAvg_data[axes+'_'+str.lower(subjectName)+'_64_mAvg.p'][:,i], label="mAvg")
# #     plt.plot(rms_data[axes+'_'+str.lower(subjectName)+'_64_rms.p'][:,i], label="rms")
# #     plt.xticks(np.arange(0, len(raw_data[axes+'_'+str.lower(subjectName)+'_raw.p'][0:105000,i]), 25000))
#     plt.legend()
#     plt.show()
# # plt.figure(figsize=(100,15))
# # plt.plot(both_aon_64_mAvg[both_aon_64_mAvg.keys()[4]])

for i in range(5):
    plt.figure(figsize=(40,10)) # (75,15) works fine too
    plt.title(header[i].split("-")[-1]+" - "+subjectToPlot+" - "+str.upper(axes))
    
    # LEFT
#     plt.plot(raw_data[axes+'_'+str.lower(subjectToPlot)+'_raw.p'][whatToPlot,i], label="Left - raw")
#     plt.plot(rect_data[axes+'_'+str.lower(subjectToPlot)+'_rectify.p'][whatToPlot,i], label="Left - rect")
#     plt.plot(mAvg_data[axes+'_'+str.lower(subjectToPlot)+'_64_mAvg.p'][whatToPlot,i], label="Left - mAvg")
    plt.plot(rms_data[axes+'_'+str.lower(subjectToPlot)+'_64_rms.p'][whatToPlot,i], label="Left - rms")
#     plt.xticks(np.arange(0, len(raw_data[axes+'_'+str.lower(subjectName)+'_raw.p'][whatToPlot,i]), 25000))

    # RIGHT
#     plt.plot(raw_data[axes+'_'+str.lower(subjectToPlot)+'_raw.p'][whatToPlot,i+5], label="Right - raw")
#     plt.plot(rect_data[axes+'_'+str.lower(subjectToPlot)+'_rectify.p'][whatToPlot,i+5], label="Right - rect")
#     plt.plot(mAvg_data[axes+'_'+str.lower(subjectToPlot)+'_64_mAvg.p'][whatToPlot,i+5], label="Right - mAvg")
    plt.plot(rms_data[axes+'_'+str.lower(subjectToPlot)+'_64_rms.p'][whatToPlot,i+5], label="Right - rms")
    plt.xticks(np.arange(0, len(raw_data[axes+'_'+str.lower(subjectToPlot)+'_raw.p'][whatToPlot,i]), 10000))

    plt.legend()
    plt.show()

In [ ]:
# tuple of (ROLL, YAW, BOTH)
slow_no_motor = {"Aon":(range(0,104000), range(0,100000), range(0,95000)),
                "Joy":(range(5000,105000), range(5000,110000), range(0,95000)),
                "Kai":(range(0,110000), range(0,145000), range(1000,115000)),
                "NNoon":(range(0,105000), range(0,100000), range(0,71000)+range(105000,145000)),
                "NNot":(range(0,115000), range(0,105000), range(0,102500))}

slow_with_motor = {"Aon":(range(104000,200000), range(100000,210000), range(100000,210000)),
                  "Joy":(range(110000,225000), range(110000,220000), range(95000,205000)),
                  "Kai":(range(115000,230000), range(145000,260000), range(140000,255000)),
                  "NNoon":(range(105000,240000), range(100000,210000), range(145000,222000)+range(248000,300000)),
                  "NNot":(range(115000,240000), range(110000,217000), range(102500,220000))}

fast_no_motor = {"Aon":(range(216000,283000), range(220000,290000), range(220000,290000)),
                "Joy":(range(270000,360000), range(280000,345000), range(235000,316000)),
                "Kai":(range(4000,95000), range(300000,405000), range(375000,470000)),
                "NNoon":(range(260000,340000), range(230000,300000), range(312500,400000)),
                "NNot":(range(247000,320000), range(217000,295000), range(220000,300000))}

fast_with_motor = {"Aon":(range(283000,350000), range(295000, 360000), range(295000,370000)),
                  "Joy":(range(370000,450000), range(350000,430000), range(330000,400000)),
                  "Kai":(range(100000,220000), range(410000,550000), range(470000,580000)),
                  "NNoon":(range(340000,435000), range(300000,402500), range(400000,480500)),
                  "NNot":(range(320000,400000), range(295000,360500), range(300000,375000))}

In [ ]:
# LOADING
# settings
subjectName = allSubject

# prepare file name
try:
    raw_filename # assume if there is no raw then no all
except  NameError:
    raw_filename, rect_filename, mAvg_filename, rms_filename = getProcessedFilename()

# create blank dictionary
raw_data, rect_data, mAvg_data, rms_data = {}, {}, {}, {}    

for tmp in subjectName:    
    loadPath = os.path.join(dataPath, processedFolderName, tmp) # path to load data
    
    # get header if not exist - channel labels, all the same
    try:
        header
    except NameError:
        header = getheader(loadPath)
    
    # extract for one subject    
    subject_filename_raw = raw_filename[tmp]
    subject_filename_rect = rect_filename[tmp]
    subject_filename_mAvg = mAvg_filename[tmp]
    subject_filename_rms = rms_filename[tmp]

    # get data
    countTb = datetime.datetime.now()

    for filename in subject_filename_raw:
        raw_data.update({filename:getdata(os.path.join(loadPath, filename))})
    for filename in subject_filename_rect:
        rect_data.update({filename:getdata(os.path.join(loadPath, filename))}) 
    for filename in subject_filename_mAvg:
        mAvg_data.update({filename:getdata(os.path.join(loadPath, filename))})
    for filename in subject_filename_rms:
        rms_data.update({filename:getdata(os.path.join(loadPath, filename))})

    countTa = datetime.datetime.now()

    print "getting data for", tmp, "takes", (countTa-countTb).seconds/60, "min", (countTa-countTb).seconds%60, "sec"
    
# temp = getdata(x[0])
# # aon = {i:getdata(i) for i in x}
# print np.shape(temp)[1]
# both_aon_64_mAvg = {i:temp[:,header.index(i)] for i in header}
# np.size(both_aon_64_mAvg[both_aon_64_mAvg.keys()[0]])
# for x in [i for i in os.listdir(loadPath) if not i.startswith(".")]:
#     print x
# x = [i for i in os.listdir(loadPath) if not i.startswith(".")]
# print x

In [ ]:
# CHECKING AND PROCESSING
# check if there is processed data
# checkProcessedIfExist()
# process and save all data
# processAll(savePath)

In [ ]:
# ----------
# run this cell and all below first
# ----------
# IMPORT
%matplotlib inline
import os
import time
import datetime
import sys
import matplotlib.pyplot as plt
import numpy as np
import cPickle
import linecache
import seaborn as sns

sns.set(style='ticks', palette='Set2')
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 1})
sns.despine()

In [ ]:
# SETTINGS VARIABLES
dataPath = "/Users/tulakan/OneDrive/BME/Rehabilitation Project/EMG data/EMG Gyro 22-23 Jul 2015"  # raw data dir
processedFolderName = "Processed"
savePath = os.path.join(dataPath, processedFolderName)
printSubjtName = True
printDataFileNameFor = "no"  # name of subject to print
windowSize = 64

# loop through directory -> get folder name for each subject and raw data file names
allSubject = [x for x in os.listdir(dataPath) if not x.startswith(".") and x != processedFolderName]
if printSubjtName: print allSubject

# data file names for each subject is collected as dictionary
dataFileName = {x: [i for i in os.listdir(os.path.join(dataPath, x)) if not i.startswith(".") and i.endswith(".ASC")]  for x in allSubject}
if printDataFileNameFor != "no":
    try:
        print dataFileName[printDataFileNameFor]
    except KeyError:
        print "No data for", printDataFileNameFor

In [ ]:
# FUNCTIONS
# check existing
def checkProcessedIfExist():
    if not os.path.exists(os.path.join(dataPath, "Processed")):
        os.makedirs(os.path.join(dataPath, "Processed"))
    for x in allSubject:
        if not os.path.exists(os.path.join(dataPath, "Processed", x)):
            os.makedirs(os.path.join(dataPath, "Processed", x))
    for x in allSubject:
        # .p is pickle files for processed data
        # this snippet trying to find .p in processed folder
        # if found it shows files name
        # else print no .p file in current dir
        if len([i for i in os.listdir(os.path.join(dataPath, processedFolderName, x)) if i.endswith(".p")]) == 0:
            print "no .p file in", os.path.join(dataPath, processedFolderName, x)
        else:
            print x, ":", [i for i in os.listdir(os.path.join(dataPath, processedFolderName, x)) if i.endswith(".p")]
        # for y in dataFileName[x]:
        # if not os.path.exists(os.path.join(dataPath, processedFolderName, x, y.split(".")[0]+"_processed.p")):
        #     print y.split(".")[0]+"_processed.p", "does not exist.."
        # print os.path.join(dataPath, allSubject[0], dataFileName[allSubject[0]][0])

# process data
def processAll(savePath):
    # count time
    startTime = datetime.datetime.now()

    for i in allSubject:
        savedir = os.path.join(savePath, i)
        print "#########################"
        print "processing", i + "..."
        print "#########################"
        for j in dataFileName[i]:
            saveProcessed(os.path.join(dataPath, i, j), savedir, windowSize)
            print "#########################"
            print j, "process - DONE"
            print "#########################"

    # savedir = os.path.join(savePath, allSubject[0])
    # processData.saveProcessed(os.path.join(dataPath, allSubject[0], dataFileName[allSubject[0]][0]), savedir, windowSize)

    # count time
    stopTime = datetime.datetime.now()
    print "data processing takes", stopTime-startTime


def saveProcessed(filename, savedir, windowSize):
    # filename = motorSpeed+actionPlane+'.ASC'
    filenameSplit = (os.path.split(filename)[-1]).split(".")[0]
    # actionname = os.path.split(filename)[-1]  # screen dir out, only pick file name
    # print filenameSplit

    # get sampling rate
    print "get sampling rate.."
    spr = linecache.getline(filename, 1)
    spr = spr.split()[0]
    print "DONE"
    # print spr
    # get header
    print "get header.."
    header = linecache.getline(filename, 3)
    header = prepareHeader(header)
    print "DONE"
    # print header
    time.sleep(1)
    # get raw data
    print "get raw data.."
    rawdt = np.loadtxt(filename, skiprows=3)
#     fstcoldt = rawdt[:, 0]  # first column data 1,2,3,...,len(data)
#     fstcoldtassec = fstcoldt/int(spr)  # convert hertz to sec
    rawdt = rawdt[:, 1:]  # 10 channels of muscles
    print "DONE"

    # process data
#     fstcoldtassec = fstcoldtassec[0:-1-windowSize+1]  # cutting out the window size
    print "rectifying data.."
    rawrecdt = abs(rawdt)  # raw data rectification
    print "DONE"
#     print "applying moving average filter.."
#     mavgdt = movingAvgFilter(windowSize, rawrecdt)  # moving average filtered
#     print "DONE"
#     print "applying root mean square filter.."
#     rmsdt = rmsFilter(windowSize, rawrecdt)  # root mean square filted
#     print "DONE"
#     rawrecdt = rawrecdt[0+(windowSize/2):-1-(windowSize/2)+1, :]  # make easier to plot

    # save data
    # savedir = os.path.join(subjectName, 'processed')
    # if not os.path.exists(savedir):
    #     os.makedirs(savedir)
    print "dumping filtered data to pickle files.."
    cPickle.dump(header, open(os.path.join(savedir, 'header.p'), "wb"))
#     cPickle.dump(fstcoldtassec, open(os.path.join(savedir, 'fstcoldtassec.p'), "wb"))
    cPickle.dump(rawdt, open(os.path.join(savedir, filenameSplit + "_raw.p"), "wb"))
    print "rawdata DONE"
    cPickle.dump(rawrecdt, open(os.path.join(savedir, filenameSplit + "_rectify.p"), "wb"))
    print "rectify DONE"
#     cPickle.dump(mavgdt, open(os.path.join(savedir, filenameSplit + "_" + str(windowSize) + "_mAvg.p"), "wb"))
#     print "moving average DONE"
#     cPickle.dump(rmsdt, open(os.path.join(savedir, filenameSplit + "_" + str(windowSize) + "_rms.p"), "wb"))
#     print "root mean square DONE"

    
def prepareHeader(rawHeader):
    ##### header manipulation
    splited = rawHeader.split()  # split header
    jointNo = 3  # number of obj to join
    loopNo = len(splited)/jointNo  # number of obj after join
    finalHeader = [splited[0]]  # pre-allocation

    for i in range(loopNo):
        finalHeader.append(splited[(3*i)+1]+splited[(3*i)+2]+splited[(3*i)+3])  # rearrange header

    return finalHeader


# filter functions

# These filters will move window over data starting at data position of (window size/2)
# so that data loss will be divided to be half for initial and half for terminal

def movingAvgFilter(windowSize, data):
    startPoint = windowSize/2  # position to start windowing
    noOfData = np.size(data, 0)
    dataCol = np.size(data, 1)

    newData = np.zeros((noOfData-windowSize, dataCol))  # -1 to ignore data[:,1]
    for i in range(startPoint, noOfData-startPoint):
        newData[i-startPoint, :] = (sum(data[i-startPoint:i+startPoint, :]))/windowSize
    return newData


def rmsFilter(windowSize, data):
    startPoint = windowSize/2  # position to start windowing
    noOfData = np.size(data, 0)
    dataCol = np.size(data, 1)

    newData = np.zeros((noOfData-windowSize, dataCol))  # -1 to ignore data[:,1]
    for i in range(startPoint, noOfData-startPoint):
        newData[i-startPoint, :] = np.sqrt(sum(data[i-startPoint:i+startPoint, :]**2))/np.sqrt(windowSize)
    return newData


# get data

def getheader(path):
    print "getting header for", subjectName + ".."
    header = cPickle.load(open((os.path.join(path, "header.p")), "rb"))
    header = header[1:len(header)]
    return header
    
def getdata(filename):
    print "getting", os.path.split(filename)[-1], "data..."
    return cPickle.load(open((os.path.join(loadPath, filename)), "rb"))

def getProcessedFilename():
    # get filename of raw and processed data for all subjects
    print "getting processed file names.."
    
    raw_filename, rect_filename, mAvg_filename, rms_filename = {}, {}, {}, {}
    for name in allSubject:
    #     print name+":",[x for x in os.listdir(os.path.join(dataPath, processedFolderName, name)) if x.endswith("raw.p")]
        raw_filename.update({name : [x for x in os.listdir(os.path.join(dataPath, processedFolderName, name)) if x.endswith("raw.p")]})
    #     print name+":",[x for x in os.listdir(os.path.join(dataPath, processedFolderName, name)) if x.endswith("rectify.p") and not x.endswith("64_rectify.p")]
        rect_filename.update({name : [x for x in os.listdir(os.path.join(dataPath, processedFolderName, name)) if x.endswith("rectify.p") and not x.endswith("64_rectify.p")]})
    #     print name+":",[x for x in os.listdir(os.path.join(dataPath, processedFolderName, name)) if x.endswith("mAvg.p")]
        mAvg_filename.update({name : [x for x in os.listdir(os.path.join(dataPath, processedFolderName, name)) if x.endswith("mAvg.p")]})
    #     print name+":",[x for x in os.listdir(os.path.join(dataPath, processedFolderName, name)) if x.endswith("rms.p")]
        rms_filename.update({name : [x for x in os.listdir(os.path.join(dataPath, processedFolderName, name)) if x.endswith("rms.p")]})

    return raw_filename, rect_filename, mAvg_filename, rms_filename
